In [35]:
import numpy as np
import pandas as pd

In [ ]:
# Load the wait time data
wait_time_df = pd.read_csv('Data/wait_times_2024_10_29.csv', na_values=['', ' '], index_col=0)

# Replace NaN values with -1 to indicate rides that are not open
wait_time_df = wait_time_df.fillna(-1)

# Replace the header with [0, 5, 10, 15, ...]
new_columns = list(i for i in range(0, 5 * wait_time_df.shape[1], 5))
wait_time_df.columns = new_columns

# Convert -1 to 0 if the next time is positive and apply linear interpolation
for ride in wait_time_df.index:
    for col in range(wait_time_df.shape[1] - 1):
        if wait_time_df.loc[ride, new_columns[col]] == -1 and wait_time_df.loc[ride, new_columns[col + 1]] > 0:
            wait_time_df.loc[ride, new_columns[col]] = 0

wait_time_df.head()

,0,5,10,15,20,25,30,35,40,45,...,935,940,945,950,955,960,965,970,975,980
Orbiter,0.0,0.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
Barnstormer,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
Big Thunder Mtn.,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,5.0,5.0,5.0,5.0,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
Buzz SRS,0.0,0.0,0.0,0.0,0.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
Dumbo,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0


In [ ]:
interpolated_df = pd.DataFrame(index=wait_time_df.index)

# Loop through each pair of consecutive columns and interpolate values in between
for i in range(len(new_columns) - 1):
    t1 = new_columns[i]
    t2 = new_columns[i + 1]
    
    mask = (wait_time_df[t1] == -1) & (wait_time_df[t2] >= 0)
    
    slope = (wait_time_df[t2] - wait_time_df[t1]) / (t2 - t1)
    intercept = wait_time_df[t1]

    for j in range(1, 5):
        interpolated_time = t1 + j
        
        # If the ride just opened, set interpolated values to 0
        interpolated_df[interpolated_time] = slope * j + intercept
        interpolated_df.loc[mask, interpolated_time] = 0

wait_time_df = pd.concat([wait_time_df, interpolated_df], axis=1).sort_index(axis=1)
wait_time_df = wait_time_df.apply(np.ceil)
wait_time_df.replace(to_replace=-1, value=10000, inplace=True)
wait_time_df.head()

/var/folders/6z/bpsd8cv959n930gw9gd94q780000gn/T/ipykernel_6237/441982089.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  interpolated_df[interpolated_time] = slope * j + intercept
/var/folders/6z/bpsd8cv959n930gw9gd94q780000gn/T/ipykernel_6237/441982089.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  interpolated_df[interpolated_time] = slope * j + intercept
/var/folders/6z/bpsd8cv959n930gw9gd94q780000gn/T/ipykernel_6237/441982089.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result o

,0,1,2,3,4,5,6,7,8,9,...,971,972,973,974,975,976,977,978,979,980
Orbiter,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,3.0,4.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
Barnstormer,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
Big Thunder Mtn.,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
Buzz SRS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
Dumbo,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0


In [38]:
wait_time_df.to_csv('Data/cleaned_wait_times_2024_10_29.csv')